In [ ]:
import numpy as np

from pydrake.all import (AddMultibodyPlantSceneGraph,
                         DiagramBuilder,
                         RigidTransform,
                         BsplineTrajectory,
                         KinematicTrajectoryOptimization,
                         SnoptSolver)

from manipulation.scenarios import (AddIiwa, AddWsg)

In [ ]:
# Joint position and velocity constraints
def make_kin_traj_opt_trajectory(q_Throw, q_Catch, v_Throw, v_Catch, t_duration):
    # Set up plant
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    iiwa = AddIiwa(plant)
    wsg = AddWsg(plant, iiwa, welded=True, sphere=True)
    gripper_frame = plant.GetFrameByName("body", wsg)
    plant.Finalize()
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    # Initialize KinTrajOpt
    num_positions = 7
    num_control_points = 21
    trajopt = KinematicTrajectoryOptimization(num_positions, num_control_points)
    prog = trajopt.get_mutable_prog()

    # Provide Initial Guess
    q_guess = np.tile(q_Catch.reshape((num_positions,1)), (1, trajopt.num_control_points()))
    q_guess[0,:] = np.linspace(q_Catch[0], q_Throw[0], trajopt.num_control_points())
    path_guess = BsplineTrajectory(trajopt.basis(), q_guess)
    trajopt.SetInitialGuess(path_guess)
    
    # Add Position and Velocity Bounds based on iiwa
    trajopt.AddPositionBounds(plant.GetPositionLowerLimits()[:num_positions],
                              plant.GetPositionUpperLimits()[:num_positions])
    trajopt.AddVelocityBounds(plant.GetVelocityLowerLimits()[:num_positions],
                              plant.GetVelocityUpperLimits()[:num_positions])
    
    # Constrain duration to be exact duration
    trajopt.AddDurationConstraint(2 * t_duration, 2 * t_duration)

    # Constrain positions in joint angles
    trajopt.AddPathPositionConstraint(q_Catch, q_Catch, 0)
    trajopt.AddPathPositionConstraint(q_Throw, q_Throw, 0.5)
    trajopt.AddPathPositionConstraint(q_Catch, q_Catch, 1)

    # Add Costs
    trajopt.AddPathLengthCost(1)

    prog.AddQuadraticErrorCost(np.eye(7), q_Catch,
                               trajopt.control_points()[:, 0])
    
    prog.AddQuadraticErrorCost(np.eye(7), q_Throw,
                               trajopt.control_points()[:, np.round(num_control_points/2).astype(int)])

    prog.AddQuadraticErrorCost(np.eye(7), q_Catch,
                               trajopt.control_points()[:, -1])

    trajopt.AddPathVelocityConstraint(v_Catch,v_Catch, 0)
    trajopt.AddPathVelocityConstraint(v_Throw,v_Throw, 0.5)
    trajopt.AddPathVelocityConstraint(v_Catch,v_Catch, 1)


    solver = SnoptSolver()
    result = solver.Solve(prog)

    if not result.is_success():
        print("Trajectory optimization failed!")
        print(result.get_solver_id().name())
    else:
        print("Trajectory optimization succeeded!")
    
    pos_traj = trajopt.ReconstructTrajectory(result)
    # vel_traj = pos_traj.MakeDerivative()
    # acc_traj = vel_traj.MakeDerivative()
    
    return pos_traj

## EvalDerivative and MakeDerivative Work Different

### MakeDerivative returns result beyond trajectory duration while EvalDerivative does not.

Our total trajectory time is 2 * t_duration. Both EvalDerivative and MakeDerivative work fine within [0, 2*t_duration].

When t > 2*t_duration, MakeDerivative still returns the result but EvalDerivative raises
an error.

Our question is why MakeDerivation still returns a result beyond the time period. If this is correct, what does that mean and why EvalDerivation raises an error?

In [ ]:
t_duration = 0.782
q_Catch = np.array([-0.74, 0.58, 0, -1.79, 0, -0.79, 0])
q_Throw = np.array([-0.54, 0.58, 0, -1.79, 0, -0.79, 0])
v_Catch = np.array([[-0.151], [3.974], [-0.121], [-0.673],[0.039], [1.305], [-0.000]])
v_Throw = np.array([[-0.151], [-4.050], [-0.121], [0.559], [0.039], [-1.322], [0.000]])

pos_traj = make_kin_traj_opt_trajectory(q_Throw, q_Catch, v_Throw, v_Catch, t_duration)

print()
print(f'Vel at {2*t_duration} using MakeDerivative')
print(pos_traj.MakeDerivative().value(2*t_duration).flatten())
print(f'Vel at {2*t_duration} using EvalDerivative')
print(pos_traj.EvalDerivative(2*t_duration, 2).flatten())

print()
print(f'Vel at {2*t_duration+0.001} using MakeDerivative')
print(pos_traj.MakeDerivative().value(2*t_duration+0.01).flatten())
print(f'Vel at {2*t_duration+0.001} using EvalDerivative')
print(pos_traj.EvalDerivative(2*t_duration+0.01, 2).flatten()) # This raises error!

Trajectory optimization succeeded!

Vel at 1.564 using MakeDerivative
[-0.09654731  2.54092072 -0.07736573 -0.43030691  0.02493606  0.83439898
  0.        ]
Vel at 1.564 using EvalDerivative
[11.894913   40.66929479 -2.83542911 -7.7167628   0.91389506 13.40725652
  0.        ]

Vel at 1.565 using MakeDerivative
[-0.09654731  2.54092072 -0.07736573 -0.43030691  0.02493606  0.83439898
  0.        ]
Vel at 1.565 using EvalDerivative


IndexError: vector::_M_range_check: __n (which is 21) >= this->size() (which is 21)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=68a898be-bd4e-4046-916a-6293aadf6c5d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>